# Data Science Term Project

## Assessing Predictive Capabilities of Various Variables in Predicting Game Newspaper Subscription

- Name: Geoff Acabado
- Student number: 59285189
- DSCI 100 003

## I. Introduction
Video games have become a widespread form of entertainment, attracting millions of players across diverse demographics (Atske & Atske, 2024). Minecraft stands out as one of the most popular sandbox games, offering an open-ended world where players can build, explore, and interact (Cipollone, Schifter, & Moffat, 2014). In addition to being a popular game, Minecraft provides a rich source of behavioral data that can be leveraged for research in areas such as player engagement, user behavior analysis, and targeted marketing. 

In this project, we analyze gameplay data collected from a Minecraft research server hosted by a research group at the University of British Columbia (UBC). The group collected detailed logs of player behavior as participants interact with the game world. This data includes various features such as age, playtime, etc., and whether or not a player has subscribed to a newsletter related to the game. 

By applying data science techniques to this data, we aim to explore whether certain player characteristics can be used to predict whether a player subscribes to the game-related newsletter. The insights from this analysis can assist the research team in better understanding the factors that drive player engagement and inform future recruitment and resource allocation strategies. **Table 1.1** shows each of the variables in the player.csv dataset and what do they mean. 

| Variable     | Type          | Description                                                                 |
|--------------|---------------|-----------------------------------------------------------------------------|
| Experience   | factor         | Experience level of player: beginner, amateur, regular, pro, veteran        |
| subscribe    | logical        | Whether the player is subscribed (TRUE/FALSE)                               |
| hashedEmail  | character      | Hashed email of player to protect privacy                                   |
| played_hours | double         | Number of hours played by players                                           |
| name         | character      | Name of the player                                                          |
| gender       | factor         | Gender identity: agender, male, female, non-binary, two-spirited, prefer not to say |
| Age          | double         | Age of the player in years                                                  |

**Table 1.1**

The table below shows a complete list of all the variables in the `player.csv` dataset used in this project.

An additional dataset contains session-level start and end time but is not utilized in this analysis as the player experience provides sufficient information for addressing the research question. **Code 0.1** contains the libraries that will be used in this project and **Code 0.2** loads the data that will be analyzed. 

In [ ]:
#code 0.1
#Some libraries to install
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 10)

# We need to set this seed so that all results in this notebook can be replicated
set.seed(9999)

In [ ]:
#Code 0.2
# install the data needed
player_data = read_csv("players.csv")
player_data

### Data explorations

In [ ]:
#Code 1.1
#Lets explore relationships between hours played, age and whether they are subscibed to game newspaper
options(repr.plot.height = 7, repr.plot.width = 10)
player_plot = player_data |>
    ggplot(aes(y = played_hours	, x = Age, color = subscribe)) +
    geom_point() +
    labs(
        title = "Hours played Vs Age of Player",
        y = "Hours played by player",
        x = "Age of player (Years)",
        colour = "Player is subscribed" ) +
    theme(text = element_text(size = 16), plot.title = element_text(hjust = 0.5))
player_plot

**Figure 1.1**

The plot above shows the relationship between the age of the player on the x-axis and the number of hours they have played Minecraft on the y-axis. Game-related newspaper subscriptions of each player are represented by the color of the points. Two datapoints are removed because the age of the player is unknown. 

The code above gave us a warning message indicating that there are 2 datapoints that have missing values or are out of range. In **code 1.15**, we show that these observations have unknown age and so were excluded from the ploting. One can see from **Figure 1.1** that most of the datapoints consist of players playing little to 0 hours in the game, making it difficult to make any inferences from the data. For this reason, the data was plotted on a logarithmic scale in **Figure 1.2** to magnify the subtleties in low values of hours played. 

In [ ]:
#Code 1.15
missing_values = player_data |>
    filter(is.na(Age))
missing_values

**Figure 1.2**

The scatter plot above shows the age of each player in the x-axis and the hours played on the y-axis but logarithmically scaled to show subtilities across different ranges. 

There was a warning that the logarithmic transformation turned the some 0s to infinity but it looks like it did not affect the purpose of our visualization that much. All of the supposedly 'negatively infinite' values remained at the bottom of the graph. From Figure 1.2, it can be easily observed that there is no obvious pattern between hours played by the player and their age. Both variables also do not seem to be correlated with whether the players are subscribed or not. However, it is important to notice that everyone who played for 10 hours or more is subscribed to a game newspaper. Another observation we see is that most players are around 15 to 25 years old, indicating that Minecraft is popular among this demographic

### II. Methods

The objective of this analysis is to find out which variables in the dataset are most predictive of the player's subscription to a game-related newspaper. We will start off by wrangling the data and giving the variables more appropriate labels. The data was first loaded and cleaned to retain only the relevant variables necessary for analysis. As shown in **Code 2.1**, we selected subscribe, played_hours, and Age from the original dataset. Since the classification model requires a categorical response variable, the subscribe column was converted into a factor. This ensured that the subsequent modeling functions would treat it appropriately as a classification target rather than a continuous variable. 

In [ ]:
#code 2.1
player_data_clean = player_data |>
    select(experience, subscribe, played_hours, gender, Age) |>
    mutate(subscribe = as.factor(subscribe)) |>
    mutate(experience = as.factor(experience)) |>
    mutate(gender = as.factor(gender)) 